In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, Row

In [5]:
spark = SparkSession.builder \
        .master("local") \
        .appName("Proyecto") \
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/28 13:50:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/28 13:50:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/28 13:50:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/28 13:50:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [9]:
path = "../../../practicas/curso-apache-spark-platzi/files/"

In [14]:
equiposSchema = StructType([
    StructField("id", IntegerType(), False),
    StructField("equipo", StringType(), False),
    StructField("sigla", StringType(), False),
])

equiposDF = spark.read \
            .schema(equiposSchema) \
            .option("header", "true") \
            .csv(path+"paises.csv")
equiposDF.show(5)


+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [24]:
SchemaJugadores = StructType([
    StructField("deportista_id", IntegerType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", IntegerType(), False),
    StructField("edad", IntegerType(), False),
    StructField("altura", IntegerType(), False),
    StructField("peso", FloatType(), False),
    StructField("equipo_id", IntegerType(), False)
])
jugadoresDF = spark.read \
            .schema(SchemaJugadores) \
            .option("header", "true") \
            .csv(path+"deportista.csv")

jugadores2DF = spark.read \
            .schema(SchemaJugadores) \
            .option("header", "true") \
            .csv(path+"deportista2.csv")
jugadoresDF = jugadoresDF.union(jugadores2DF)
jugadoresDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [22]:
resultadosSchema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False),
])

resultadosDF = spark.read \
            .schema(resultadosSchema) \
            .option("header", "true") \
            .csv(path+"resultados.csv")
resultadosDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [28]:
deporteSchema = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False)
])


deportesDF = spark.read \
             .schema(deporteSchema) \
             .option("header", "true")\
             .csv(path+"deporte.csv")
deportesDF.take(2)


[Row(deporte_id=1, deporte='Basketball'), Row(deporte_id=2, deporte='Judo')]

In [34]:
eventoSchema = StructType([
    StructField("evento_id", IntegerType(), False),
    StructField("evento", StringType(), False),
    StructField("deporte_id", IntegerType(), False)
])

eventosDF = spark.read\
           .schema(eventoSchema) \
           .option("header", "true") \
           .csv(path+"evento.csv")

eventosDF.take(5)
del eventoDF

In [33]:
eventosDF.printSchema()

root
 |-- evento_id: integer (nullable = true)
 |-- evento: string (nullable = true)
 |-- deporte_id: integer (nullable = true)



### Eliminar y renombrar columnas en un dataframe 

In [35]:
jugadoresDF = jugadoresDF.withColumnRenamed("genero", "sexo").drop("altura")

In [54]:
jugadoresDF.printSchema()

root
 |-- deportista_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- EdadAlJugar: integer (nullable = true)
 |-- equipo_id: integer (nullable = true)



In [44]:
from pyspark.sql.functions import col

In [55]:
try: 
    jugadoresDF=jugadoresDF.select(col("deportista_id"),col("nombre"), col("edadAlJugar"), col("equipo_id"))
    jugadoresDF.show(5)
except:
    print("No existe la columna")

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [56]:
jugadoresDF.sort("edadAlJugar").show(10)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|           54|Mohamed Jamshid A...|          0|      496|
|           58|    Georgi Abadzhiev|          0|      154|
|           66|     Mohamed Abakkar|          0|     1003|
|          133|           Franz Abb|          0|      399|
|          102|   Sayed Fahmy Abaza|          0|      308|
|          139|George Ioannis Abbot|          0|     1043|
|          163|     Ismail Abdallah|          0|     1095|
|          167|Ould Lamine Abdallah|          0|      362|
|          173| Mohamed Abdel Fatah|          0|     1003|
|          176|Mahmoud Atter Abd...|          0|     1095|
+-------------+--------------------+-----------+---------+
only showing top 10 rows



22/03/28 16:02:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 67787, Lee BongJu, 27, 970
 Schema: deportista_id, nombre, edad, equipo_id
Expected: deportista_id but found: 67787
CSV file: file:///home/facturatech/Documentos/practicas/curso-apache-spark-platzi/files/deportista2.csv


In [59]:
jugadoresDF.sort("edadAlJugar").filter((jugadoresDF.edadAlJugar !=0)).show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        52070|        Etsuko Inada|         11|      514|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        47618|Sonja Henie Toppi...|         11|      742|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



22/03/28 16:07:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 67787, Lee BongJu, 27, 970
 Schema: deportista_id, nombre, edad, equipo_id
Expected: deportista_id but found: 67787
CSV file: file:///home/facturatech/Documentos/practicas/curso-apache-spark-platzi/files/deportista2.csv
